In [35]:
import pandas as pd
import numpy as np
from scipy import stats
import os 

def perform_ks_tests_and_save(model_name, sae_path, esl_path, aae_path, output_csv_path):
    # Load data from text files
    SAE = pd.read_csv(sae_path, sep="	", header=None)
    ESL = pd.read_csv(esl_path, sep="	", header=None)
    AAE = pd.read_csv(aae_path, sep="	", header=None)
    
    # Convert dataframes to numpy arrays
    sample_sae = SAE[1].to_numpy()
    sample_esl = ESL[1].to_numpy()
    sample_aae = AAE[1].to_numpy()
    
    # Perform KS tests
    result_sae_esl = stats.ks_2samp(sample_sae, sample_esl)
    result_sae_aae = stats.ks_2samp(sample_sae, sample_aae)
    
    # Prepare data for CSV
    data = [
        [model_name, "AES_Score", "SAE-ESL", result_sae_esl.statistic, result_sae_esl.pvalue],
        [model_name, "AES_Score", "SAE-AAE", result_sae_aae.statistic, result_sae_aae.pvalue]
    ]
    
    # Create DataFrame
    df = pd.DataFrame(data, columns=["Model", "Metric", "Pair", "Statistic", "P-Value"])
    
    # Check if the CSV file already exists to decide on adding a header
    file_exists = os.path.isfile(output_csv_path)
    
    # Save to CSV, append if file exists, include header if file does not exist
    df.to_csv(output_csv_path, mode='a', index=False, header=not file_exists)
    
    print(f"Results saved to {output_csv_path}")


In [44]:
perform_ks_tests_and_save(
    "Alpaca",
    "case2_EducationalDisparity/results/MultiScaleBertAESResults/alpaca_base_results.txt",
    "case2_EducationalDisparity/results/MultiScaleBertAESResults/alpaca_ESL_results.txt",
    "case2_EducationalDisparity/results/MultiScaleBertAESResults/alpaca_AAE_results.txt",
    "ks_test_results_case2.csv"
)


Results saved to ks_test_results.csv


In [48]:
alpaca = pd.read_csv('case2_EducationalDisparity/all_essays_alpaca.csv', sep=';')

In [51]:
alpaca.metrics_SAE[0]

"{'flesch_reading_ease': 61.67, 'smog_index': 11.5, 'flesch_kincaid_grade': 9.1, 'unique_words': 141, 'lexical_diversity': 0.4533762057877814, 'grammar_errors': 1}"

In [53]:
import pandas as pd
import numpy as np
from scipy import stats
import ast  # For converting string representation of a dictionary to a dictionary
import os  # To check if the output CSV file already exists

def perform_ks_tests_and_append_results(model_name, input_csv_path, metric, output_csv_path):
    # Load data from the CSV file
    data_df = pd.read_csv(input_csv_path, sep=';')
    
    # Convert string representations of dictionaries into actual dictionaries
    data_df['metrics_SAE'] = data_df['metrics_SAE'].apply(ast.literal_eval)
    data_df['metrics_AAE'] = data_df['metrics_AAE'].apply(ast.literal_eval)
    data_df['metrics_ESL'] = data_df['metrics_ESL'].apply(ast.literal_eval)
    
    # Extract the specified metric into numpy arrays
    sample_sae = np.array([d[metric] for d in data_df['metrics_SAE']])
    sample_aae = np.array([d[metric] for d in data_df['metrics_AAE']])
    sample_esl = np.array([d[metric] for d in data_df['metrics_ESL']])
    
    # Perform KS tests
    result_sae_esl = stats.ks_2samp(sample_sae, sample_esl)
    result_sae_aae = stats.ks_2samp(sample_sae, sample_aae)
    
    # Prepare data for CSV
    data = [
        [model_name, metric, "SAE-ESL", result_sae_esl.statistic, result_sae_esl.pvalue],
        [model_name, metric, "SAE-AAE", result_sae_aae.statistic, result_sae_aae.pvalue]
    ]
    
    # Check if output CSV file exists; if so, append. Otherwise, create a new file.
    if os.path.exists(output_csv_path):
        df_existing = pd.read_csv(output_csv_path)
        df_new = pd.DataFrame(data, columns=["Model", "Metric", "Pair", "Statistic", "P-Value"])
        df_updated = pd.concat([df_existing, df_new], ignore_index=True)
        df_updated.to_csv(output_csv_path, index=False)
    else:
        df = pd.DataFrame(data, columns=["Model", "Metric", "Pair", "Statistic", "P-Value"])
        df.to_csv(output_csv_path, index=False)
    
    print(f"Results appended to {output_csv_path}")


In [83]:
perform_ks_tests_and_append_results(
    "Gemini",
    "case2_EducationalDisparity/all_essays_gemini.csv",
    "lexical_diversity",
    "ks_test_results_case2.csv"
)

Results appended to ks_test_results.csv


In [ ]:
'flesch_reading_ease'
'lexical_diversity'
'grammar_errors'


In [84]:
results = pd.read_csv('ks_test_results.csv')

In [85]:
filtered_rows = results[results["P-Value"] < 0.05]
# Count the number of rows
num_rows = len(filtered_rows)
print(f"Number of rows where pvalue is less than 0.05: {num_rows}")
print(f"Total number of rows: ", len(results))

Number of rows where pvalue is less than 0.05: 31
Total number of rows:  64


In [15]:
import pandas as pd
import numpy as np
from scipy import stats
import ast  # For converting string representation of a dictionary to a dictionary
import os  # To check if the output CSV file already exists

def perform_t_test_and_append_results(model_name, input_csv_path, metric, output_csv_path):
    # Load data from the CSV file
    data_df = pd.read_csv(input_csv_path, sep=';')
    
    # Convert string representations of dictionaries into actual dictionaries
    data_df['metrics_SAE'] = data_df['metrics_SAE'].apply(ast.literal_eval)
    data_df['metrics_AAE'] = data_df['metrics_AAE'].apply(ast.literal_eval)
    data_df['metrics_ESL'] = data_df['metrics_ESL'].apply(ast.literal_eval)
    
    # Extract the specified metric into numpy arrays
    sample_sae = np.array([d[metric] for d in data_df['metrics_SAE']])
    sample_aae = np.array([d[metric] for d in data_df['metrics_AAE']])
    sample_esl = np.array([d[metric] for d in data_df['metrics_ESL']])
    
    # Perform KS tests
    result_sae_esl = stats.ttest_ind(sample_sae, sample_esl)
    result_sae_aae = stats.ttest_ind(sample_sae, sample_aae)
    
    # Prepare data for CSV
    data = [
        [model_name, metric, "SAE-ESL", result_sae_esl.statistic, result_sae_esl.pvalue],
        [model_name, metric, "SAE-AAE", result_sae_aae.statistic, result_sae_aae.pvalue]
    ]
    
    # Check if output CSV file exists; if so, append. Otherwise, create a new file.
    if os.path.exists(output_csv_path):
        df_existing = pd.read_csv(output_csv_path)
        df_new = pd.DataFrame(data, columns=["Model", "Metric", "Pair", "Statistic", "P-Value"])
        df_updated = pd.concat([df_existing, df_new], ignore_index=True)
        df_updated.to_csv(output_csv_path, index=False)
    else:
        df = pd.DataFrame(data, columns=["Model", "Metric", "Pair", "Statistic", "P-Value"])
        df.to_csv(output_csv_path, index=False)
    
    print(f"Results appended to {output_csv_path}")


In [39]:
perform_t_test_and_append_results(
    "Vicuna13B",
    "case2_EducationalDisparity/all_essays_vicuna13B.csv",
    "flesch_reading_ease",
    "t_test_results_case2.csv"
)

#'flesch_reading_ease'
#'lexical_diversity'
#'grammar_errors'

Results appended to t_test_results_case2.csv


In [41]:
import pandas as pd
import numpy as np
from scipy import stats
import os 

def perform_t_tests_and_save(model_name, sae_path, esl_path, aae_path, output_csv_path):
    # Load data from text files
    SAE = pd.read_csv(sae_path, sep="	", header=None)
    ESL = pd.read_csv(esl_path, sep="	", header=None)
    AAE = pd.read_csv(aae_path, sep="	", header=None)
    
    # Convert dataframes to numpy arrays
    sample_sae = SAE[1].to_numpy()
    sample_esl = ESL[1].to_numpy()
    sample_aae = AAE[1].to_numpy()
    
    # Perform KS tests
    result_sae_esl = stats.ttest_ind(sample_sae, sample_esl)
    result_sae_aae = stats.ttest_ind(sample_sae, sample_aae)
    
    # Prepare data for CSV
    data = [
        [model_name, "AES_Score", "SAE-ESL", result_sae_esl.statistic, result_sae_esl.pvalue],
        [model_name, "AES_Score", "SAE-AAE", result_sae_aae.statistic, result_sae_aae.pvalue]
    ]
    
    # Create DataFrame
    df = pd.DataFrame(data, columns=["Model", "Metric", "Pair", "Statistic", "P-Value"])
    
    # Check if the CSV file already exists to decide on adding a header
    file_exists = os.path.isfile(output_csv_path)
    
    # Save to CSV, append if file exists, include header if file does not exist
    df.to_csv(output_csv_path, mode='a', index=False, header=not file_exists)
    
    print(f"Results saved to {output_csv_path}")


In [49]:
perform_t_tests_and_save(
    "Vicuna7B",
    "case2_EducationalDisparity/results/MultiScaleBertAESResults/vicuna7B_base_results.txt",
    "case2_EducationalDisparity/results/MultiScaleBertAESResults/vicuna7B_ESL_results.txt",
    "case2_EducationalDisparity/results/MultiScaleBertAESResults/vicuna7B_AAE_results.txt",
    "t_test_results_case2.csv"
)


Results saved to t_test_results_case2.csv


In [1]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('t_test_two_sided_results_case2.csv')  # Replace 'your_file.csv' with your CSV file path


threshold = 0.016666 # 0.05 / 3


results = {'SAE-AAE': {}, 'SAE-ESL': {}}


for pair in ['SAE-AAE', 'SAE-ESL']:

    filtered_df = df[df['Pair'] == pair]
    
    # Group by model and check if any p-value is below the threshold for each model
    for model, group in filtered_df.groupby('Model'):
        significant = 'significant' if any(group['P-Value'] < threshold) else 'not significant'
        results[pair][model] = significant

# Save the results
with open('t_test_significance_results_case2.txt', 'w') as file:
    for pair, models in results.items():
        file.write(f"Results for {pair}:\n")
        for model, result in models.items():
            file.write(f"{model}: {result}\n")
        file.write("\n")


